In [1]:
import pandas as pd 
import numpy as np
import json
import os
import json
import gzip
from nltk.tokenize import word_tokenize
import gensim
from gensim.models import word2vec
from sklearn.feature_extraction.text import TfidfVectorizer

In [2]:
data = []

with open(r"C:\Users\Rishabh\Downloads\Amazon_Project_Data\Beauty\meta_All_Beauty.jsonl (1).gz", "rt", encoding="utf-8") as fp:
    for line in fp:
        data_line = json.loads(line.strip())
        data.append(data_line)

In [3]:
df = pd.DataFrame(data)

In [4]:
np.random.seed(42)

In [5]:
df = df.sample(frac = 0.5).reset_index()

In [6]:
df.columns

Index(['index', 'main_category', 'title', 'average_rating', 'rating_number',
       'features', 'description', 'price', 'images', 'videos', 'store',
       'categories', 'details', 'parent_asin', 'bought_together'],
      dtype='object')

In [7]:
df['org_title'] = df['title'].copy()

In [8]:
## Lower The strings ##
df['title'] = df['title'].str.lower() 

In [12]:
def remove_punct(string):
    for i in string:
        puncts = '''!()-[]{};:'",<>./?@#$%^&*_~'''
        if i in puncts:
           string = string.replace(i,'')
            
    return string

In [10]:
df['title'] = df['title'].apply(lambda x: remove_punct(x)) 

In [14]:
import nltk
nltk.download('punkt_tab')

df['title'] = df['title'].apply(lambda x: word_tokenize(x)) 

[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\Rishabh\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt_tab.zip.


In [15]:
df['title'].sample(5)

32399    [windmax, heated, mitten, spa, electric, warmi...
49460    [efaster, baby, spoon, shower, bath, water, sw...
16997    [kitty, kuromi, melody, hair, clips, japanese,...
4900     9545hp, dressing, tegaderm, hp, frame, lf, ...
45941    [coisini, hair, brazilian, virgin, hair, body,...
Name: title, dtype: object

In [16]:
from nltk.corpus import stopwords
stopwords = stopwords.words('english')

In [17]:
def remove_stopwords(text):
    for i in text:
        if i in stopwords:
            text.remove(i)
    return text

In [18]:
df['title'] = df['title'].apply(lambda x: remove_stopwords(x)) 

In [19]:
model = gensim.models.Word2Vec(df['title'],min_count = 1)

In [20]:
from sklearn.metrics.pairwise import cosine_similarity

def similarity(title_1,title_2):

    ## Extract the learned word embeddings after training  
    title_1_tokens  = [i for i in title_1 if i in model.wv]
    title_2_tokens  = [i for i in title_2 if i in model.wv]

    ## average the word embeddings vector to convert to fixed-size vectors
    if not title_1_tokens or not title_2_tokens:  # Handle cases where no words are found in the model's vocabulary
        return 0

    
    title_1_vec = np.mean(model.wv[title_1_tokens],axis = 0)
    title_2_vec = np.mean(model.wv[title_2_tokens],axis = 0)

    ## Calculate Similarity between 2 vectors (titles) 
    similarity = cosine_similarity([title_1_vec],[title_2_vec])[0][0]
    return similarity
    

In [21]:
title_1 = df['org_title'][0]

title_2 = df['org_title'][100]

In [22]:
similarity(title_1,title_2)

0.96127796

In [23]:
title_1 = df['title'].iloc[0]
title_2 = df['title'].iloc[2]

title_1_tokens  = [i for i in title_1 if i in model.wv]
title_2_tokens  = [i for i in title_2 if i in model.wv]

title_1_vec = np.mean(model.wv[title_1_tokens],axis = 0)
title_1_vec = np.mean(model.wv[title_2_tokens],axis = 0)

for i in range(len(df[0:10000])):
    sims  = similarity(title_1,df['title'].iloc[i])


def top_similar_titles(title,top_n):    
    similar_scores = sorted([(similarity(title,df['title'].iloc[i]),i) for i in range(len(df[100:10000]))])[0:top_n]
    similar_indices = [i[1] for i in similar_scores]

    
    return similar_indices 

In [24]:
sim = top_similar_titles(df['title'].iloc[34],10) 
sim

[2028, 9608, 7720, 4441, 9670, 5310, 5878, 9208, 9740, 8170]

In [25]:
for a in sim:
    print(df['images'].iloc[a][0]['large'])

https://m.media-amazon.com/images/I/41gNBBWzGRL.jpg
https://m.media-amazon.com/images/I/316fEVYeJ0L.jpg
https://m.media-amazon.com/images/I/41XRJhOUpQL.jpg
https://m.media-amazon.com/images/I/51HJh7jzwRL.jpg
https://m.media-amazon.com/images/I/41j2rvRciDS.jpg
https://m.media-amazon.com/images/I/01RmK+J4pJL.gif
https://m.media-amazon.com/images/I/31tKCdQmHSL.jpg
https://m.media-amazon.com/images/I/51sxaLUtjJL.jpg
https://m.media-amazon.com/images/I/51v5lIagRBL.jpg
https://m.media-amazon.com/images/I/51x1QcU9crL.jpg


In [26]:
for a in sim:
    print(df['org_title'].iloc[a])

Stylemate

TimeTinkle
SLKIJDHFB
Vovotrade
ASTARON
PINX
LanLan butterfly flower ring earring
UNA Dreadlocks Beads Hair Braid Rings Clips Dread Locks Hair Braiding Metal Cuffs Decoration/Accessories Jewelry (50piece/pack,Golden)
Aprince Bobby Pins Bobby Pins Buns U Shaped Pins for Women Gold Hair Pins Bobby Pins Open Flower Bobby Pins 200 Pieces Gold


In [27]:
## Using Fiass for Large Datasets

# import faiss


# title_1_tokens  = [i for i in title_1 if i in model.wv]
# title_2_tokens  = [i for i in title_2 if i in model.wv]

# title_1_vec = np.mean(model.wv[title_1_tokens],axis = 0)
# title_2_vec = np.mean(model.wv[title_2_tokens],axis = 0)

# faiss.normalize_L2(title_1_vec) 
# faiss.normalize_L2(title_2_vec)

# index = faiss.IndexFlatIP(title_1_vec.shape[1])
# index.add(title_2_vec)  # Add title_2_vec to FAISS index

# D, I = index.search(title_1_vec, k=2) 

# print(f"Cosine Similarity: {D[0][0]}")  # Similarity score
# print(f"Closest vector index: {I[0][0]}")  # Index of closest match

In [ ]:
# import faiss
# import numpy as np

# # Example vectors (title_1_vec and title_2_vec)
# title_1_vec = np.array([[0.1, 0.2, 0.3]], dtype=np.float32)  # Shape (1, D)
# title_2_vec = np.array([[0.2, 0.1, 0.4]], dtype=np.float32)  # Shape (1, D)

# # Normalize vectors using FAISS
# faiss.normalize_L2(title_1_vec)  # In-place normalization
# faiss.normalize_L2(title_2_vec)

# # Create FAISS index (Inner Product = Cosine Similarity)
# index = faiss.IndexFlatIP(title_1_vec.shape[1])
# index.add(title_2_vec)  # Add title_2_vec to FAISS index

# # Query similarity
# D, I = index.search(title_1_vec, k=1)  # Find most similar vector

# print(f"Cosine Similarity: {D[0][0]}")  # Similarity score
# print(f"Closest vector index: {I[0][0]}")  # Index of closest match
